In [141]:
import requests

from database import *

In [125]:
ACCEPTED_JSON_TYPES = ['application/json', 'application/geo+json']

def get_api_collections(base_api):
    """
    Returns the collections endpoint from the API base URL
    For example:
    https://api.hamburg.de/datasets/v1/fahrradhaeuschen/
    returns
    https://api.hamburg.de/datasets/v1/fahrradhaeuschen/collections
    """
    # the header we send with our requests
    headers = {'Content-Type': 'application/json'}

    base_response = requests.get(base_api, headers=headers)

    collection_links = []

    # check if the request was successful
    if base_response.status_code == 200:

        base_json = base_response.json()
        base_links = base_json['links']

        # find the link with the rel 'data'
        for base_link in base_links:
            if base_link['rel'] == 'data':
                collections_api = base_link['href']

                collections_response = requests.get(collections_api, headers=headers)

                # check if the request was successful
                if collections_response.status_code == 200:
                    collections_json = collections_response.json()

                    return collections_json['collections']

def get_items_endpoint(base_api):
    """
    Returns the items endpoint from the API base URL
    For example:
    https://api.hamburg.de/datasets/v1/fahrradhaeuschen/
    returns
    https://api.hamburg.de/datasets/v1/fahrradhaeuschen/collections/items?f=json

    :param base_api: the base URL of the API
    :return: the items endpoint or None if the items endpoint could not be found
    """

    # the header we send with our requests
    headers = {'Content-Type': 'application/json'}

    base_response = requests.get(base_api, headers=headers)

    item_links = []

    # check if the request was successful
    if base_response.status_code == 200:

        base_json = base_response.json()
        base_links = base_json['links']

        # find the link with the rel 'data'
        for base_link in base_links:
            if base_link['rel'] == 'data':
                collections_api = base_link['href']

                collections_response = requests.get(collections_api, headers=headers)

                # check if the request was successful
                if collections_response.status_code == 200:
                    collections_json = collections_response.json()

                    collections = collections_json['collections']

                    for collection in collections:
                        collection_links = collection['links']

                        # find the link with the rel 'items'
                        # and the type 'application/json' or 'application/geo+json'
                        for collection_link in collection_links:
                            if collection_link['rel'] == 'items' and collection_link['type'] in ACCEPTED_JSON_TYPES:
                                item_links.append(collection_link['href'])
    
    # one of the requests failed
    return item_links

In [122]:
links = [
    'https://api.hamburg.de/datasets/v1/perspektive_wohnen',
    'https://api.hamburg.de/datasets/v1/schulen',
    'https://api.hamburg.de/datasets/v1/pflegeeinrichtungen',
    'https://api.hamburg.de/datasets/v1/krankenhaeuser',
    'https://api.hamburg.de/datasets/v1/notunterkuenfte',
    'https://api.hamburg.de/datasets/v1/waermekataster_stromverbrauch',
    'https://api.hamburg.de/datasets/v1/feuerwehrstandorte',
    'https://api.hamburg.de/datasets/v1/deichinformationen',
    'https://api.hamburg.de/datasets/v1/wasserrettungspunkte',
    'https://api.hamburg.de/datasets/v1/uesg',
    'https://api.hamburg.de/datasets/v1/deichinformationen',
    'https://api.hamburg.de/datasets/v1/hvv_einzugsbereiche'
]

In [126]:
get_api_collections(links[0])

[{'title': 'Bestehende Flüchtlingsunterkünfte mit der Perspektive Wohnen',
  'id': 'perspektive_wohnen_bestehend',
  'extent': {'spatial': {'bbox': [[9.770895066763483,
      53.47530752046935,
      10.139275162346298,
      53.67115976219237]],
    'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'}},
  'itemType': 'feature',
  'crs': ['#/crs'],
  'storageCrs': 'http://www.opengis.net/def/crs/EPSG/0/25832',
  'links': [{'rel': 'items',
    'type': 'text/html',
    'title': "Access the features in the collection 'Bestehende Flüchtlingsunterkünfte mit der Perspektive Wohnen' as HTML",
    'href': 'https://api.hamburg.de/datasets/v1/perspektive_wohnen/collections/perspektive_wohnen_bestehend/items?f=html'},
   {'rel': 'items',
    'type': 'application/geo+json',
    'title': "Access the features in the collection 'Bestehende Flüchtlingsunterkünfte mit der Perspektive Wohnen' as GeoJSON",
    'href': 'https://api.hamburg.de/datasets/v1/perspektive_wohnen/collections/perspektive_wohnen

In [123]:
for link in links:
    print(get_items_endpoint(link))
    print()

['https://api.hamburg.de/datasets/v1/perspektive_wohnen/collections/perspektive_wohnen_bestehend/items?f=json', 'https://api.hamburg.de/datasets/v1/perspektive_wohnen/collections/perspektive_wohnen_geplant/items?f=json']

['https://api.hamburg.de/datasets/v1/schulen/collections/hh_schulen_eingaenge/items?f=json', 'https://api.hamburg.de/datasets/v1/schulen/collections/nicht_staatliche_schulen/items?f=json', 'https://api.hamburg.de/datasets/v1/schulen/collections/staatliche_schulen/items?f=json']

['https://api.hamburg.de/datasets/v1/pflegeeinrichtungen/collections/vollstationaere_pflegeeinrichtungen/items?f=json']

['https://api.hamburg.de/datasets/v1/krankenhaeuser/collections/gesundheit_krankenhaeuser/items?f=json']

['https://api.hamburg.de/datasets/v1/notunterkuenfte/collections/notunterkuenfte/items?f=json']

['https://api.hamburg.de/datasets/v1/waermekataster_stromverbrauch/collections/stromverbrauch/items?f=json']

['https://api.hamburg.de/datasets/v1/feuerwehrstandorte/collecti

In [106]:
def request_dataset(dataset, verbose=False):
    """
    Takes in a Dataset object from the database and requests the dataset from the API.
    Returns the JSON response from the API or None if the request failed.
    """

    url = dataset.url
    items_link = get_items_endpoint(url)

    if items_link is not None:
        response = requests.get(items_link)
        if response.status_code == 200:
            response_json = response.json()

            if verbose: print(f'{response_json["numberReturned"]} items returned from {response_json["totalFeatures"]}')
    
    return None

In [142]:
engine, session = connect_db()

# read all urls from urls.txt
with open('urls.txt', 'r') as urls_file:
    # strip the newline character from each line
    urls = [url.strip() for url in urls_file.readlines()]

for url in urls:

    base_response = requests.get(url).json()

    title = base_response['title']
    description = base_response['description']

    dataset = Dataset(
        title=title,
        description=description,
        url=url
        )

    

    collections = get_api_collections(url)
    
    for collection in collections:
        print(collection['title'])
        print(collection['id'])

Bestehende Flüchtlingsunterkünfte mit der Perspektive Wohnen
perspektive_wohnen_bestehend
Geplante Flüchtlingsunterkünfte mit der Perspektive Wohnen
perspektive_wohnen_geplant
Eingänge der Schulen Hamburgs
hh_schulen_eingaenge
Nicht Staatliche Schulen Hamburg
nicht_staatliche_schulen
Staatliche Schulen Hamburg
staatliche_schulen
Vollstationäre Pflegeeinrichtungen
vollstationaere_pflegeeinrichtungen
gesundheit_krankenhaeuser
gesundheit_krankenhaeuser
Notunterkuenfte
notunterkuenfte
Absoluter Stromverbrauch im Gebäudecluster
stromverbrauch
Berufsfeuerwehr
berufsfeuerwehr
Freiwillige Feuerwehr
freiwillige_feuerwehr
Bauwerksspezifikationen
bauwerksspezifikationen
Grundgrenze Deich
deichgrundgrenze
Deichkilometer
deichkilometer
Deichverteidigung
deichverteidigung
Deichlinie/Hauptdeichlinie
hauptdeichlinie
Wasserrettungspunkte
wasserrettungspunkte
Überschwemmungsgebiete
ueberschwemmungsgebiete
Blattschnitte
uesg_blattschnitte
Bauwerksspezifikationen
bauwerksspezifikationen
Grundgrenze Deich
